In [1]:
import pandas as pd
import scanpy as sc
import squidpy as sq
import numpy as np
import os
from scipy.io import mmread

/users/asanb3/.local/lib/python3.9/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/users/asanb3/.local/lib/python3.9/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
# --- 1. Setup paths, groups, and cluster keys ---
input_dir = "/data/salomonis2/LabFiles/Shunya_Asanuma/Spatial/LungChat/Output/Xenium/GSE250346/04.squidpy/metadata/"
output_file = "/data/salomonis2/LabFiles/Shunya_Asanuma/Spatial/LungChat/Output/Xenium/GSE250346/04.squidpy/combined_nhood_enrichment_multi_key.xlsx"
group_ids = ["More_Affected", "Less_Affected", "Unaffected"]
cluster_keys = ["TNiche", "final_CT"] 
all_results = {} # Dictionary to store all resulting DataFrames

# --- 2. Loop through each group and each cluster key ---
for group_id in group_ids:
    print(f"--- Processing group: {group_id} ---")

    # Load data for the current group
    counts_path = f"{input_dir}/{group_id}_counts.mtx"
    if not os.path.exists(counts_path):
        print(f"Warning: Data for group '{group_id}' not found. Skipping.")
        continue

    counts = mmread(counts_path).T
    metadata = pd.read_csv(f"{input_dir}/{group_id}_metadata.csv", index_col=0)
    spatial_coords = pd.read_csv(f"{input_dir}/{group_id}_spatial_coords.csv", index_col=0)
    genes = pd.read_csv(f"{input_dir}/{group_id}_gene_names.tsv", header=None, sep="\t")[0].values
    barcodes = pd.read_csv(f"{input_dir}/{group_id}_cell_barcodes.tsv", header=None, sep="\t")[0].values

    # Assemble the AnnData object
    adata = sc.AnnData(X=counts, obs=metadata, var=pd.DataFrame(index=genes))
    adata.obs.index = barcodes
    adata.obsm['spatial'] = spatial_coords.loc[adata.obs_names, :].values
    adata.obs['sample'] = adata.obs['sample'].astype('category')
    
    # Pre-compute the spatial neighbors once per group
    sq.gr.spatial_neighbors(adata, coord_type="generic", spatial_key="spatial", library_key="sample")

    # Inner loop for cluster keys
    for cluster_key in cluster_keys:
        print(f"  ... analyzing with cluster_key: '{cluster_key}'")
        
        if cluster_key not in adata.obs.columns:
            print(f"Warning: Column '{cluster_key}' not found in metadata for group '{group_id}'. Skipping.")
            continue
        adata.obs[cluster_key] = adata.obs[cluster_key].astype('category')
        
        # Compute neighborhood enrichment
        sq.gr.nhood_enrichment(adata, cluster_key=cluster_key)

        # Extract the raw z-score matrix
        uns_key = f"{cluster_key}_nhood_enrichment"
        zscore_matrix = adata.uns[uns_key]["zscore"]
        cell_type_labels = adata.obs[cluster_key].cat.categories
        zscore_df = pd.DataFrame(zscore_matrix, index=cell_type_labels, columns=cell_type_labels)
        
        # --- NEW: Perform signed log10 transformation ---
        transformed_df = np.sign(zscore_df) * np.log10(np.abs(zscore_df) + 1)
        
        # Store both raw and transformed results in the dictionary
        raw_key = f"{group_id}_{cluster_key}_raw_zscore"
        transformed_key = f"{group_id}_{cluster_key}_log_zscore"
        all_results[raw_key] = zscore_df
        all_results[transformed_key] = transformed_df

print("\n--- All combinations processed. ---")

--- Processing group: More_Affected ---
  ... analyzing with cluster_key: 'TNiche'


  0%|          | 0/1000 [00:00<?, ?/s]

  ... analyzing with cluster_key: 'final_CT'


  0%|          | 0/1000 [00:00<?, ?/s]

/users/asanb3/.local/lib/python3.9/site-packages/squidpy/gr/_nhood.py:202: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


...

  0%|          | 0/1000 [00:00<?, ?/s]

  ... analyzing with cluster_key: 'final_CT'


  0%|          | 0/1000 [00:00<?, ?/s]


--- All combinations processed. ---


/users/asanb3/.local/lib/python3.9/site-packages/squidpy/gr/_nhood.py:202: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [9]:
# --- 3. Save all results to a single HDF5 file ---
output_hdf_file = "/data/salomonis2/LabFiles/Shunya_Asanuma/Spatial/LungChat/Output/Xenium/GSE250346/04.squidpy/metadata/h5/combined_nhood_enrichment.h5"

# NEW: Create the output directory if it doesn't exist
output_dir = os.path.dirname(output_hdf_file)
os.makedirs(output_dir, exist_ok=True)

# Use the 'w' (write) mode to create a new file each time
with pd.HDFStore(output_hdf_file, mode='w') as store:
    for result_key, df_to_save in all_results.items():
        store.put(result_key, df_to_save)
        print(f"Saved results for '{result_key}' to HDF5 file.")

print(f"\n Successfully saved all analyses to: {output_hdf_file}")

Saved results for 'More_Affected_TNiche_raw_zscore' to HDF5 file.
Saved results for 'More_Affected_TNiche_log_zscore' to HDF5 file.
Saved results for 'More_Affected_final_CT_raw_zscore' to HDF5 file.
Saved results for 'More_Affected_final_CT_log_zscore' to HDF5 file.
Saved results for 'Less_Affected_TNiche_raw_zscore' to HDF5 file.
Saved results for 'Less_Affected_TNiche_log_zscore' to HDF5 file.
Saved results for 'Less_Affected_final_CT_raw_zscore' to HDF5 file.
Saved results for 'Less_Affected_final_CT_log_zscore' to HDF5 file.
Saved results for 'Unaffected_TNiche_raw_zscore' to HDF5 file.
Saved results for 'Unaffected_TNiche_log_zscore' to HDF5 file.
Saved results for 'Unaffected_final_CT_raw_zscore' to HDF5 file.
Saved results for 'Unaffected_final_CT_log_zscore' to HDF5 file.

 Successfully saved all analyses to: /data/salomonis2/LabFiles/Shunya_Asanuma/Spatial/LungChat/Output/Xenium/GSE250346/04.squidpy/metadata/h5/combined_nhood_enrichment.h5
